### Exploratory Data Analysis with SQL ###

In [5]:
!pip install pandas
!pip install -U scikit-learn
!pip install matplotlib
!pip install numpy
!pip install seaborn
!pip install plotly
!pip install requests
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install ipython-sql prettytable
!pip install -q pandas

%load_ext sql
%sql sqlite:///my_data1.db
    
import pandas as pd
import requests
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

  Using cached SQLAlchemy-1.3.9.tar.gz (6.0 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  ERROR: Error [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software while executing command python setup.py egg_info
ERROR: Could not install packages due to an OSError: [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software



The sql extension is already loaded. To reload it, use:
  %reload_ext sql


#### Read from Data Source and Write to SQL Tables ####

In [10]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")
%sql DROP TABLE IF EXISTS SPACEXTABLE;
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null
print(df.head())

 * sqlite:///my_data1.db
Done.
 * sqlite:///my_data1.db
Done.
         Date Time (UTC) Booster_Version  Launch_Site  \
0  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  PAYLOAD_MASS__KG_  \
0               Dragon Spacecraft Qualification Unit                  0   
1  Dragon demo flight C1, two CubeSats, barrel of...                  0   
2                              Dragon demo flight C2                525   
3                                       SpaceX CRS-1                500   
4                                       SpaceX CRS-2                677   

       Orbit         Customer Mission_Outcome      Landing_Outcome  
0        LEO           SpaceX         Success  Failure (parachute)  
1  LEO

#### Task 1: Display the names of the unique launch sites in the space mission ####

In [17]:
query1 = "SELECT DISTINCT Launch_Site FROM SPACEXTABLE;"
cur.execute(query1)

launch_sites = cur.fetchall()
for site in launch_sites:
    print(site[0])  # Print each unique launch site

CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


#### Task 2: Display 5 records where launch sites begin with the string 'CCA' ####

In [20]:
query2 = "SELECT * FROM SPACEXTABLE WHERE Launch_Site LIKE 'CCA%' LIMIT 5;"
cur.execute(query2)

records = cur.fetchall()
for record in records:
    print(record)

('2010-06-04', '18:45:00', 'F9 v1.0  B0003', 'CCAFS LC-40', 'Dragon Spacecraft Qualification Unit', 0, 'LEO', 'SpaceX', 'Success', 'Failure (parachute)')
('2010-12-08', '15:43:00', 'F9 v1.0  B0004', 'CCAFS LC-40', 'Dragon demo flight C1, two CubeSats, barrel of Brouere cheese', 0, 'LEO (ISS)', 'NASA (COTS) NRO', 'Success', 'Failure (parachute)')
('2012-05-22', '7:44:00', 'F9 v1.0  B0005', 'CCAFS LC-40', 'Dragon demo flight C2', 525, 'LEO (ISS)', 'NASA (COTS)', 'Success', 'No attempt')
('2012-10-08', '0:35:00', 'F9 v1.0  B0006', 'CCAFS LC-40', 'SpaceX CRS-1', 500, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')
('2013-03-01', '15:10:00', 'F9 v1.0  B0007', 'CCAFS LC-40', 'SpaceX CRS-2', 677, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')


#### Task 3: Display the total payload mass carried by boosters launched by NASA (CRS) ####

In [23]:
query3 = """
SELECT SUM(PAYLOAD_MASS__KG_) AS Total_Payload_Mass
FROM SPACEXTABLE
WHERE Customer = 'NASA (CRS)';
"""
cur.execute(query3)
total_payload_mass = cur.fetchone()[0]
print(f"Total Payload Mass for NASA (CRS): {total_payload_mass} kg")

Total Payload Mass for NASA (CRS): 45596 kg


#### Task 4: Display average payload mass carried by booster version F9 v1.1 ####

In [26]:
query4 = """
SELECT AVG(PAYLOAD_MASS__KG_) AS Average_Payload_Mass
FROM SPACEXTABLE
WHERE Booster_Version = 'F9 v1.1';
"""
cur.execute(query4)
average_payload_mass = cur.fetchone()[0]
print(f"Average Payload Mass for F9 v1.1: {average_payload_mass} kg")

Average Payload Mass for F9 v1.1: 2928.4 kg


#### Task 5: List the date when the first succesful landing outcome in ground pad was achieved #### 

In [29]:
query5 = """
SELECT MIN(Date) AS First_Successful_Landing
FROM SPACEXTABLE
WHERE Landing_Outcome LIKE '%ground pad%' AND Mission_Outcome = 'Success';
"""
cur.execute(query5)
first_successful_landing = cur.fetchone()[0]
print(f"First successful landing on ground pad: {first_successful_landing}")

First successful landing on ground pad: 2015-12-22


#### Task 6: List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000 ####

In [32]:
query6 = """
SELECT Booster_Version
FROM SPACEXTABLE
WHERE Landing_Outcome LIKE '%drone ship%'
AND Mission_Outcome = 'Success'
AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;
"""
cur.execute(query6)
boosters = cur.fetchall()
for booster in boosters:
    print(booster[0])

F9 FT B1020
F9 FT B1022
F9 FT B1026
F9 FT  B1021.2
F9 FT  B1031.2


#### Task 7: List the total number of successful and failure mission outcomes ####

In [36]:
query7 = """
SELECT Mission_Outcome, COUNT(*) AS Total_Count
FROM SPACEXTABLE
GROUP BY Mission_Outcome;
"""
cur.execute(query7)
mission_outcomes = cur.fetchall()
for outcome, count in mission_outcomes:
    print(f"{outcome}: {count} missions")

Failure (in flight): 1 missions
Success: 98 missions
Success : 1 missions
Success (payload status unclear): 1 missions


#### Task 8: List all the booster_versions that have carried the maximum payload mass, using a subquery ####

In [40]:
query8 = """
SELECT Booster_Version
FROM SPACEXTABLE
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE);
"""
cur.execute(query8)
boosters = cur.fetchall()
for booster in boosters:
    print(booster[0]) 

F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2 
F9 B5 B1058.3 
F9 B5 B1051.6
F9 B5 B1060.3
F9 B5 B1049.7 


#### Task 9: List the records which will display the month names, failure landing_outcomes in drone ship, booster versions, launch_site for the months in year 2015 ####

In [43]:
query9 = """
SELECT 
    CASE substr(Date, 6, 2)
        WHEN '01' THEN 'January'
        WHEN '02' THEN 'February'
        WHEN '03' THEN 'March'
        WHEN '04' THEN 'April'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'June'
        WHEN '07' THEN 'July'
        WHEN '08' THEN 'August'
        WHEN '09' THEN 'September'
        WHEN '10' THEN 'October'
        WHEN '11' THEN 'November'
        WHEN '12' THEN 'December'
    END AS Month_Name,
    Booster_Version,
    Launch_Site,
    Landing_Outcome
FROM SPACEXTABLE
WHERE substr(Date, 0, 5) = '2015'
AND Landing_Outcome LIKE '%drone ship%'
AND Landing_Outcome LIKE '%Failure%';
"""
cur.execute(query9)
records9 = cur.fetchall()
for record in records9:
    print(record)

('January', 'F9 v1.1 B1012', 'CCAFS LC-40', 'Failure (drone ship)')
('April', 'F9 v1.1 B1015', 'CCAFS LC-40', 'Failure (drone ship)')


#### Task 10: Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order ####

In [46]:
query10 = """
SELECT Landing_Outcome, COUNT(*) AS Outcome_Count
FROM SPACEXTABLE
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY Landing_Outcome
ORDER BY Outcome_Count DESC;
"""
cur.execute(query10)
landing_outcomes = cur.fetchall()
for outcome, count in landing_outcomes:
    print(f"{outcome}: {count} landings")

No attempt: 10 landings
Success (drone ship): 5 landings
Failure (drone ship): 5 landings
Success (ground pad): 3 landings
Controlled (ocean): 3 landings
Uncontrolled (ocean): 2 landings
Failure (parachute): 2 landings
Precluded (drone ship): 1 landings
